In [ ]:
pip install transformers

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
import torch

In [ ]:
p_df = pd.read_csv("positive.csv")
n_df = pd.read_csv("negative.csv")

In [ ]:
p_df

In [ ]:
n_df

In [ ]:
def get_training_data(p_df,n_df):
  
  train_texts = [*p_df['text'] , *n_df['text']]
  train_labels = [*p_df['label'] , *n_df['label']]
  
  return train_texts, train_labels


In [ ]:
train_texts,train_labels = get_training_data(p_df,n_df)

In [ ]:

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [ ]:

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
import torch
class BanglaDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = BanglaDataset(train_encodings, train_labels)
val_dataset = BanglaDataset(val_encodings, val_labels)


In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

In [ ]:
trainer.save_model("/content/test")

In [ ]:
test_model = DistilBertForSequenceClassification.from_pretrained("/content/results/checkpoint-500")
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

loading configuration file /content/results/checkpoint-500/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "vocab_size": 30522
}

loading weights file /content/results/checkpoint-500/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at /content/results

In [ ]:
nlp = pipeline('sentiment-analysis', model=test_model, tokenizer=tokenizer)

In [ ]:
results = nlp(["আমার জীবনের সবথেকে ফালতু একটা নাটক এইটা"])

In [ ]:
for result in results:
  print(result)

{'label': 'LABEL_0', 'score': 0.9052820801734924}


In [ ]:
trainer.evaluate(val_dataset)

***** Running Evaluation *****
  Num examples = 2362
  Batch size = 64


{'epoch': 3.0,
 'eval_loss': 0.23698624968528748,
 'eval_runtime': 37.5094,
 'eval_samples_per_second': 62.971,
 'eval_steps_per_second': 0.986}